# Problem 1:

In [92]:
from math import sin
import numpy as np
import matplotlib.pyplot as plt 
import numpy.polynomial.polynomial as poly

In [115]:
def createX(n, degree, x, y):
    onesArr = np.ones((n, 1))
    x = x.reshape(n, 1)
    for degree in range(1, degree+1):
        if degree == 1:
            X = np.hstack((onesArr, x))
        else:
            X = np.hstack((X, x**degree))
    return X

def calBeta(X, y):
    X_TX = np.matmul(np.transpose(X), X)
    X_TX_inv = np.linalg.pinv(X_TX)
    X_Ty = np.matmul(np.transpose(X), y)
    beta = np.matmul(X_TX_inv, X_Ty)
    return beta

In [ ]:
numOfSamples = 51
x = np.linspace(0, 5 , numOfSamples)
e = np.random.normal(0, 1, numOfSamples)
y = x + e

X = createX(numOfSamples, 11, x, y)
beta = calBeta(X, y)
print(np.shape(X))
y_pred = np.matmul(X, beta)
# print(y_pred)
plt.scatter(x, y)
plt.plot(x, y_pred, 'black')

coefs = poly.polyfit(x, y, 10)
ffit = poly.polyval(x, coefs)
# plt.plot(x, ffit, 'red')
plt.show()

# Problem 3

In [1]:
import pandas as pd

In [6]:
mydata= pd.read_table("spambase/spambase.data", header=none)

NameError: name 'none' is not defined

In [5]:
print(mydata)

     0,0.64,0.64,0,0.32,0,0,0,0,0,0,0.64,0,0,0,0.32,0,1.29,1.93,0,0.96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.778,0,0,3.756,61,278,1
0     0.21,0.28,0.5,0,0.14,0.28,0.21,0.07,0,0.94,0.2...                                                                                                    
1     0.06,0,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,0....                                                                                                    
2     0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31,0...                                                                                                    
3     0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31,0...                                                                                                    
4     0,0,0,0,1.85,0,0,1.85,0,0,0,0,0,0,0,0,0,0,0,0,...                                                                                                    
5     0,0,0,0,1.92,0,0,0,0,0.64,0.96,1.28,0,0,0,0.96...         